# LBM-CaImAn Pipeline Demo

This notebook demonstrates the new unified pipeline API for processing calcium imaging data with CaImAn.

In [1]:
import lbm_caiman_python as lcp
from mbo_utilities import imread
import numpy as np
from pathlib import Path

print(f"lbm_caiman_python version: {lcp.__version__}")

lbm_caiman_python version: 0.1.0


## 1. Setup Paths

Set your input data path and output directory.

In [2]:
# path to your input data (tiff file or directory)
input_path = Path(r"E:\datasets\lbm\jeff_lbm")

# output directory for results
output_path = Path(r"E:\datasets\lbm\jeff_lbm\caiman")

print(f"Input: {input_path}")
print(f"Output: {output_path}")

Input: E:\datasets\lbm\jeff_lbm
Output: E:\datasets\lbm\jeff_lbm\caiman


In [3]:
# load raw scanimage tiff(s) as a lazy array
data = imread(input_path)
print(f"Shape: {data.shape}")
print(f"Dtype: {data.dtype}")

# lazy arrays carry scanimage metadata
if hasattr(data, "metadata"):
    meta = data.metadata
    print(f"Frame rate: {meta.get('fr', 'N/A')} Hz")
    print(f"Num planes: {meta.get('num_planes', 'N/A')}")

Shape: (8440, 2, 1002, 725)
Dtype: int16
Frame rate: 4.69 Hz
Num planes: 2


## 2. View Default Parameters

Check the default CaImAn parameters.

In [4]:
# view all default parameters
ops = lcp.default_ops()

print("Motion Correction Parameters:")
for k, v in ops.items():
    if k.startswith("do_motion") or k in ["max_shifts", "strides", "overlaps", "pw_rigid", "gSig_filt"]:
        print(f"  {k}: {v}")

print("\nCNMF Parameters:")
for k, v in ops.items():
    if k.startswith("do_cnmf") or k in ["K", "gSig", "gSiz", "p", "merge_thresh", "min_SNR", "rval_thr"]:
        print(f"  {k}: {v}")

print("\nGeneral Parameters:")
for k, v in ops.items():
    if k in ["fr", "n_processes", "dxy"]:
        print(f"  {k}: {v}")

Motion Correction Parameters:
  do_motion_correction: True
  max_shifts: (6, 6)
  strides: (48, 48)
  overlaps: (24, 24)
  pw_rigid: True
  gSig_filt: (2, 2)

CNMF Parameters:
  do_cnmf: True
  K: 50
  gSig: (4, 4)
  gSiz: None
  p: 1
  merge_thresh: 0.8
  min_SNR: 2.5
  rval_thr: 0.85

General Parameters:
  fr: 30.0
  n_processes: None
  dxy: (1.0, 1.0)


## 3. Customize Parameters (Optional)

Modify parameters as needed for your data.

In [5]:
# start with defaults
ops = lcp.default_ops()

# customize for your data
ops["fr"] = meta["frame_rate"]             # frame rate in Hz
ops["dxy"] = (meta["dx"], meta["dy"])             # frame rate in Hz
ops["gSig"] = (2, 2)          # expected neuron half-width in pixels
ops["K"] = 400                # expected number of neurons
ops["min_SNR"] = 2          # minimum SNR for accepting components
ops["rval_thr"] = 0.7        # correlation threshold

# motion correction settings
ops["do_motion_correction"] = False    # maximum shift in pixels

ops["max_shifts"] = (6, 6)    # maximum shift in pixels
ops["pw_rigid"] = True        # use piecewise-rigid correction
ops["strides"] = (48, 48)     # patch size for pw-rigid
ops["overlaps"] = (24, 24)    # patch overlap

print("Custom parameters set.")

Custom parameters set.


## 4. Run Pipeline

Run the full processing pipeline (motion correction + CNMF).

In [ ]:
# run the pipeline
results = lcp.pipeline(
    input_data=input_path,
    save_path=output_path,
    ops=ops,
    planes=[1, 2],    # uncomment to process specific planes
    force_mcorr=False,   # set True to re-run motion correction
    force_cnmf=False,    # set True to re-run CNMF
)

print(f"\nProcessed {len(results)} plane(s)")
for r in results:
    print(f"  {r}")

Loading input: E:\datasets\lbm\jeff_lbm
Detected 4D input, delegating to run_volume...
Processing 2 planes (total: 2)
Output: E:\datasets\lbm\jeff_lbm\caiman

--- Plane 1/2 ---
  Output: E:\datasets\lbm\jeff_lbm\caiman\zplane01_tp00001-08440
  Writing input movie...
  Running CNMF...
  CNMF failed: Unable to allocate 22.8 GiB for an array with shape (1002, 725, 8440) and data type float32


Traceback (most recent call last):
  File "C:\Users\RBO\repos\LBM-CaImAn-Python\lbm_caiman_python\run_lcp.py", line 636, in run_plane
    cnmf_result = _run_cnmf(movie_for_cnmf, ops, plane_dir)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\RBO\repos\LBM-CaImAn-Python\lbm_caiman_python\run_lcp.py", line 764, in _run_cnmf
    cnmf.fit(movie)
  File "C:\Users\RBO\repos\LBM-CaImAn-Python\.pixi\envs\default\Lib\site-packages\caiman\source_extraction\cnmf\cnmf.py", line 363, in fit
    self.initialize(Y)
  File "C:\Users\RBO\repos\LBM-CaImAn-Python\.pixi\envs\default\Lib\site-packages\caiman\source_extraction\cnmf\cnmf.py", line 725, in initialize
    initialize_components(Y, sn=estim.sn, options_total=self.params.to_dict(),
  File "C:\Users\RBO\repos\LBM-CaImAn-Python\.pixi\envs\default\Lib\site-packages\caiman\source_extraction\cnmf\initialization.py", line 334, in initialize_components
    Ain, Cin, _, b_in, f_in = greedyROI(
                              ^^


--- Plane 2/2 ---
  Output: E:\datasets\lbm\jeff_lbm\caiman\zplane02_tp00001-08440
  Writing input movie...
  Running CNMF...


## 5. Load and Inspect Results

In [ ]:
# load results from first plane
if results:
    ops_path = results[0]
    plane_dir = ops_path.parent

    # load all results
    data = lcp.load_planar_results(plane_dir)

    print(f"Loaded results from: {plane_dir}")
    print(f"\nAvailable data:")
    for key in data.keys():
        val = data[key]
        if isinstance(val, np.ndarray):
            print(f"  {key}: shape={val.shape}, dtype={val.dtype}")
        elif isinstance(val, dict):
            print(f"  {key}: dict with {len(val)} keys")
        else:
            print(f"  {key}: {type(val).__name__}")

In [ ]:
# get processing summary
if results:
    ops = lcp.load_ops(results[0])

    print(f"Plane: {ops.get('plane', '?')}")
    print(f"Number of cells: {ops.get('n_cells', '?')}")
    print(f"Frame rate: {ops.get('fr', '?')} Hz")
    print(f"Image size: {ops.get('Ly', '?')} x {ops.get('Lx', '?')}")
    print(f"Number of frames: {ops.get('nframes', '?')}")

    # show processing history
    if "processing_history" in ops:
        print("\nProcessing history:")
        for step in ops["processing_history"]:
            print(f"  {step['step']}: {step.get('duration_seconds', '?')}s")

## 6. Visualize Results

In [ ]:
import matplotlib.pyplot as plt

if results:
    data = lcp.load_planar_results(results[0].parent)

    # plot mean image with contours
    if "template" in data:
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(data["template"], cmap="gray")
        ax.set_title("Mean Image (Motion Correction Template)")
        ax.axis("off")
        plt.show()

In [ ]:
# plot fluorescence traces
if results and "F" in data:
    F = data["F"]
    n_cells = min(10, F.shape[0])

    fig, axes = plt.subplots(n_cells, 1, figsize=(14, 2*n_cells), sharex=True)
    if n_cells == 1:
        axes = [axes]

    for i, ax in enumerate(axes):
        ax.plot(F[i], "k", linewidth=0.5)
        ax.set_ylabel(f"Cell {i}")
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)

    axes[-1].set_xlabel("Frame")
    fig.suptitle("Raw Fluorescence Traces")
    plt.tight_layout()
    plt.show()

In [ ]:
# plot dF/F traces
if results and "dff" in data:
    dff = data["dff"]
    n_cells = min(10, dff.shape[0])

    fig, axes = plt.subplots(n_cells, 1, figsize=(14, 2*n_cells), sharex=True)
    if n_cells == 1:
        axes = [axes]

    for i, ax in enumerate(axes):
        ax.plot(dff[i], "b", linewidth=0.5)
        ax.set_ylabel(f"Cell {i}")
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)

    axes[-1].set_xlabel("Frame")
    fig.suptitle("dF/F Traces")
    plt.tight_layout()
    plt.show()

## 7. Get Accepted/Rejected Cells

In [ ]:
if results:
    accepted, rejected = lcp.get_accepted_cells(results[0].parent)

    print(f"Accepted cells: {len(accepted)}")
    print(f"Rejected cells: {len(rejected)}")

    if len(accepted) > 0:
        print(f"\nAccepted indices: {accepted[:10]}..." if len(accepted) > 10 else f"\nAccepted indices: {accepted}")

## 8. Compute ROI Statistics

In [ ]:
if results:
    stats = lcp.compute_roi_stats(results[0].parent)

    print("ROI Statistics:")
    for k, v in stats.items():
        if isinstance(v, (int, float)):
            print(f"  {k}: {v:.3f}" if isinstance(v, float) else f"  {k}: {v}")
        elif isinstance(v, np.ndarray):
            print(f"  {k}: array of shape {v.shape}")

## 9. Alternative: Run Only Motion Correction or CNMF

In [ ]:
# run only motion correction (skip CNMF)
ops_mcorr_only = lcp.default_ops()
ops_mcorr_only["do_cnmf"] = False

# results_mcorr = lcp.pipeline(
#     input_data=input_path,
#     save_path=output_path / "mcorr_only",
#     ops=ops_mcorr_only,
# )

In [ ]:
# run only CNMF (skip motion correction, assumes pre-registered data)
ops_cnmf_only = lcp.default_ops()
ops_cnmf_only["do_motion_correction"] = False

# results_cnmf = lcp.pipeline(
#     input_data=input_path,
#     save_path=output_path / "cnmf_only",
#     ops=ops_cnmf_only,
# )

## 10. Using Arrays Directly

You can also pass lazy arrays or numpy arrays directly to the pipeline.

In [ ]:
# load raw scanimage tiffs as a lazy array
# arr = imread(input_path)
# print(f"Shape: {arr.shape}")
# print(f"Dtype: {arr.dtype}")

# pass lazy array directly to pipeline
# results = lcp.pipeline(
#     input_data=arr,
#     save_path=output_path,
#     ops=ops,
# )

# or convert to numpy first if needed
# np_arr = np.asarray(arr)
# results = lcp.pipeline(
#     input_data=np_arr,
#     save_path=output_path,
#     ops=ops,
# )